<h1 style="text-align: center; font-weight: bold; font-size: 36px;">Makemore Part 4: Backprop Ninja</h1>

# Introduction

Manual backprop through a **MLP** model. Pen-and-paper derivations

Inspired by Karpathy [Neural Networks: Zero-to-Hero](https://github.com/karpathy/nn-zero-to-hero). 
We are using the same [names.txt](https://github.com/karpathy/makemore/blob/master/names.txt) as in Zero to Hero so we can compare results.

# Imports

In [2]:
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Build the Dataset

In [3]:
with open('../data/names.txt', 'r') as f:
    names = f.read().splitlines()
print("Num names:", len(names))
print("Example names:", names[:10])
print("Min length:", min(len(name) for name in names))
print("Max length:", max(len(name) for name in names))

Num names: 32033
Example names: ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']
Min length: 2
Max length: 15


In [4]:
# Get vocabulary
letters = sorted(list(set(''.join(names))))
letters = ['.'] + letters
n_vocab = len(letters)
print(letters)

['.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
class Tokenizer:
    def __init__(self, vocab):
        assert isinstance(vocab, list)
        assert all(isinstance(v, str) for v in vocab)
        assert all(len(v) == 1 for v in vocab)
        self.stoi = {ch: i for i, ch in enumerate(vocab)}
        self.itos = {i: ch for i, ch in enumerate(vocab)}

    def encode(self, text):
        return [self.stoi[s] for s in text]

    def decode(self, sequence):
        if isinstance(sequence, list):
            return ''.join([self.itos[i] for i in sequence])
        elif isinstance(sequence, torch.Tensor):
            assert sequence.ndim in [0, 1]
            if sequence.ndim == 0:
                return self.itos[sequence.item()]  # one char
            else:
                return ''.join([self.itos[i.item()] for i in sequence])
        else:
            raise ValueError(f"Type {type(sequence)} not supported")

In [6]:
def build_dataset(tok, block_size, names):
    X, Y = [], []  # inputs and targets
    for name in names:
        name = '.'*block_size + name + '.'  # add start/stop tokens '..emma.'
        for i in range(len(name) - block_size):
            X.append(tok.encode(name[i:i+block_size]))
            Y.append(tok.encode(name[i+block_size])[0])  # [0] to keep Y 1d tensor
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

In [7]:
block_size = 3  # context length
tok = Tokenizer(vocab=letters)

random.seed(42)
random.shuffle(names)
n1 = int(0.8*len(names))
n2 = int(0.9*len(names))

Xtr, Ytr = build_dataset(tok, block_size, names[:n1])
Xval, Yval = build_dataset(tok, block_size, names[n1:n2])
Xtest, Ytest = build_dataset(tok, block_size, names[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [8]:
def cmp(s, at, bt):
    ex = torch.all(at == bt).item()
    app = torch.allclose(at, bt)
    maxdiff = (at - bt).abs().max().item()
    print(f'{s:18s} | exat: {str(ex):5s} | approx: {str(app):5s} | maxdiff: {maxdiff}')

# Train the Model

In [12]:
# Init Layers
torch.manual_seed(0)

# Hyperparameters
n_batch = 32
n_embd = 10
n_hid = 64

# Model
C = torch.randn((n_vocab, n_embd))                              # n_vocab, n_emb (embeddings)
W1_kaiming_init = (5/3)/((n_embd*block_size)**0.5)              # tanh_gain / sqrt(fan_in)
W1 = torch.randn((n_embd*block_size, n_hid)) * W1_kaiming_init  # n_seq*n_emb, n_hid
b1 = torch.randn(n_hid)                      * 0.1              # n_hid
bngain = torch.randn((1, n_hid))             * 0.1 + 1.0        # 1, n_hid
bnbias = torch.randn((1, n_hid))             * 0.1              # 1, n_hid
W2 = torch.randn((n_hid, n_vocab))           * 0.1              # n_hid, n_out
b2 = torch.randn(n_vocab)                    * 0.1              # 1, n_out

# Gather Params
params = [C, W1, b1, bngain, bnbias, W2, b2]
for p in params:
    p.requires_grad = True

In [13]:
iters, losses = [], []

lr_schedule = [0.1]*100000 + [0.01]*100000
num_epochs = len(lr_schedule)
batch_size = 32
i = 0

In [14]:
for _ in range(num_epochs):

    # Random mini batch
    batch_indices = torch.randint(0, Xtr.shape[0], (batch_size,))
    x_batch = Xtr[batch_indices]
    y_batch = Ytr[batch_indices]

    # Forward Pass

    # Embedding
    emb = C[x_batch]                                  # n_batch, n_seq, n_emb
    embcat = emb.view(-1, n_embd*block_size)          # n_batch, n_embd*block_size

    # Linear 1
    z1 = embcat @ W1 + b1                             # n_batch, n_hid

    # Bachnorm 1
    # zx = (x - x_mean) / (x_var + 1e-5)**0.5
    # bn(x) = zx * gain + bias
    # Batchnorm 1 - mean
    z1_sum = z1.sum(dim=0, keepdim=True)              # 1, n_hid
    z1_mean = z1_sum / n_batch                        # 1, n_hid
    # Batchnorm 1 - var
    z1_diff = z1 - z1_mean                            # n_batch, n_hid
    z1_diff_2 = z1_diff**2.0                          # n_batch, n_hid
    z1_diff_sum = z1_diff_2.sum(dim=0, keepdim=True)  # 1, n_hid
    z1_var = z1_diff_sum * (n_batch-1)**-1            # 1, n_hid
    # Batchnorm 1 - correction
    z1_var_p_eps = z1_var + 1e-5                      # 1, n_hid
    z1_var_sqrt_inv = z1_var_p_eps**-0.5              # 1, n_hid
    zx = z1_diff * z1_var_sqrt_inv                    # n_batch, n_hid
    # Batchnorm 1 - scaling
    zz = bngain * zx + bnbias                         # n_batch, n_hid

    # Tanh 1
    # tanh(x) = (torch.exp(2*x) - 1.0) / (torch.exp(2*x) + 1)
    zz_2 = 2*zz                                 # n_batch, n_hid
    zz_2_exp = zz_2.exp()                       # n_batch, n_hid
    zz_2_exp_m1 = zz_2_exp - 1.0                # n_batch, n_hid
    zz_2_exp_p1 = zz_2_exp + 1.0                # n_batch, n_hid
    zz_2_exp_p1_inv = zz_2_exp_p1**-1           # n_batch, n_hid
    h1 = zz_2_exp_m1 * zz_2_exp_p1_inv          # n_batch, n_hid

    # Linear 2
    logits = h1 @ W2 + b2                       # n_batch, n_vocab

    # Softmax
    logits_max = logits.max(dim=-1, keepdim=True).values      # n_batch, 1
    logits_diff = logits - logits_max                         # n_batch, n_vocab
    logits_exp = logits_diff.exp()                            # n_batch, n_vocab
    logits_exp_sum = logits_exp.sum(dim=-1, keepdim=True)     # n_batch, 1
    logits_exp_sum_inv = logits_exp_sum**-1                   # n_batch, 1
    probs = logits_exp * logits_exp_sum_inv                   # n_batch, n_vocab

    # Cross Entropy
    log_probs = probs.log()                                         # n_batch, n_vocab
    log_probs_target = log_probs[range(n_batch), y_batch]           # n_batch
    loss = -log_probs_target.sum(dim=0, keepdim=True) / n_batch

    # Backward Pass
    loss.backward()

    with torch.no_grad():
        
        # Cross Entropy
        d_loss = torch.tensor(1.0)
        d_log_probs_target = -1 * torch.ones(n_batch) * (1/n_batch * d_loss)
        d_log_probs = torch.zeros_like(log_probs)
        d_log_probs[range(n_batch), y_batch] = d_log_probs_target

        # Softmax
        d_probs = (1 / probs) * d_log_probs    # deriv of ln * chain rule
        d_logits_exp_sum_inv = (logits_exp * d_probs).sum(dim=-1, keepdim=True)
        d_logits_exp_sum = -1 * logits_exp_sum**-2 * d_logits_exp_sum_inv
        d_logits_exp = torch.ones_like(logits_exp) * d_logits_exp_sum
        d_logits_exp += logits_exp_sum_inv * d_probs  # second gradient path
        d_logits_diff = logits_exp * d_logits_exp
        d_logits_max = -d_logits_diff.sum(dim=-1, keepdim=True)   # n_batch, 1
        d_logits = d_logits_diff.clone()  # logits_diff path
        logits_maxi = logits.max(dim=-1).indices
        d_logits[range(n_batch), logits_maxi] += d_logits_max[range(n_batch), 0]   # max path

        # Linear 2
        d_h1 = d_logits @ W2.T
        d_W2 = h1.T @ d_logits
        d_b2 = d_logits.sum(dim=0)

        # Tanh 1
        d_zz_2_exp_p1_inv = zz_2_exp_m1 * d_h1
        d_zz_2_exp_p1 = -1 * zz_2_exp_p1**-2 * d_zz_2_exp_p1_inv
        d_zz_2_exp_m1 = zz_2_exp_p1_inv * d_h1
        d_zz_2_exp = d_zz_2_exp_p1 + d_zz_2_exp_m1
        d_zz_2 = zz_2_exp * d_zz_2_exp
        d_zz = 2 * d_zz_2

        # Batch Norm 1 - scaling
        d_bngain = (zx * d_zz).sum(dim=0, keepdim=True)
        d_bnbias = d_zz.sum(dim=0, keepdim=True)
        # Batch Norm 1 - correction
        d_zx = torch.ones_like(zx) * bngain * d_zz
        d_z1_var_sqrt_inv = (z1_diff * d_zx).sum(dim=0, keepdim=True)
        d_z1_var_p_eps = -0.5*z1_var_p_eps**-1.5 * d_z1_var_sqrt_inv
        # Batch Norm 1 - var
        d_z1_var = d_z1_var_p_eps.clone()
        d_z1_diff_sum = (n_batch-1)**-1 * d_z1_var
        d_z1_diff_2 = torch.ones_like(z1_diff_2) * d_z1_diff_sum
        d_z1_diff = 2 * z1_diff * d_z1_diff_2
        d_z1_diff += d_zx * z1_var_sqrt_inv  # second path
        # Batch Norm 1 - mean
        d_z1_mean = -1 * d_z1_diff.sum(dim=0, keepdim=True)
        d_z1_sum = 1/n_batch * d_z1_mean
        d_z1 = torch.ones_like(z1) * d_z1_sum
        d_z1 += d_z1_diff

        # Linear 1
        d_embcat = d_z1 @ W1.T
        d_W1 = embcat.T @ d_z1
        d_b1 = d_z1.sum(dim=0)

        # Embedding
        d_emb = d_embcat.view(-1, block_size, n_embd)
        d_C = torch.zeros_like(C)
        d_C.index_add_(0, x_batch.view(-1), d_emb.view(-1, n_embd))


    # Stats
    lr = lr_schedule[i]
    for p in params:
        p.data += -lr * p.grad

    if i % 10000 == 0:
        print(i, loss.item())

    iters.append(i)
    losses.append(loss.item())
    i += 1

    with torch.no_grad():
        # PyTorch loss: 3.3894288539886475
        print(f"PyTorch loss: {F.cross_entropy(logits, y_batch).item()}")
        print(f"              {loss.item()}")

    break

0 3.3894288539886475
PyTorch loss: 3.3894288539886475
              3.3894288539886475


In [15]:
# Gather Params
# params = [C, W1, b1, bngain, bnbias, W2, b2]
grads = [d_C, d_W1, d_b1, d_bngain, d_bnbias, d_W2, d_b2]
names = ["d_C", "d_W1", "d_b1", "d_bngain", "d_bnbias", "d_W2", "d_b2"]
for param, grad, name in zip(params, grads, names):
    cmp(name, grad, param.grad)

d_C                | exat: True  | approx: True  | maxdiff: 0.0
d_W1               | exat: True  | approx: True  | maxdiff: 0.0
d_b1               | exat: True  | approx: True  | maxdiff: 0.0
d_bngain           | exat: True  | approx: True  | maxdiff: 0.0
d_bnbias           | exat: True  | approx: True  | maxdiff: 0.0
d_W2               | exat: True  | approx: True  | maxdiff: 0.0
d_b2               | exat: True  | approx: True  | maxdiff: 0.0
